In [1]:
%%capture
!pip install stanza

In [2]:
%%capture
import stanza
stanza.download('es')
nlp = stanza.Pipeline('es')

In [3]:
file1 = open('dic-judeo-español.txt', 'r', encoding="utf-8")
Lines = file1.readlines()
dic=[]
for l in Lines:
  p = {"src":l.split(";")[0],"target":l.split(";")[1]}
  dic.append(p)

In [4]:
import re, string

def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [5]:
def remove_punctuation (s):
  return re.sub('[%s]' % re.escape(string.punctuation), ' ', s)

In [6]:
def judeo_parse(s):
  if s.find("h") != -1:
    indices = [i for i, x in enumerate(s) if x == "h"]
    flag_h = 0
    for h in indices:
      string_list = list(s)
      if h != 0:
        if s[h-2] != "c":
          string_list[h] = ""
      else:
        string_list[0] = ""
    s = "".join(string_list)
  if s.find("y") != -1:
    if len(s) == 1:
      s = s.replace("y","i")
  if s.find("ca") != -1:
    s = s.replace("ca","ka")
  if s.find("co") != -1:
    s = s.replace("co","ko")
  if s.find("ñ") != -1:
    s = s.replace("ñ","ny")
  if s.find("qu") != -1:
    s = s.replace("qu","k")
  if s.find("rd") != -1:
    s = s.replace("rd","dr")
  if s.find("cr") != -1:
    s = s.replace("cr","kr")
  if s.find("ll") != -1:
    s = s.replace("ll","y")
  if s.find("g") != -1:
    s = s.replace("g","j")
  return s

In [71]:
def complete_word(mayor,menor):
  for l in range(len(mayor)-len(menor)):
    menor = menor+" "
  return menor

def get_suffix(a,b):
  flag = 0
  mayor = ""
  menor = ""

  if len(a) > flag:
    mayor = a
    menor = b
    flag = len(a)
  if len(b) > flag:
    mayor = b
    menor = a
  menor = complete_word(mayor,menor)

  f = []
  for x, y in zip(mayor,menor):
    if x == y:
      f.append(x)

  h = ''.join([str(elem) for elem in f])

  for l in range(len(a)-len(h)):
    h = h+" "

  p = []
  for x, y in zip(a,h):
    if x != y:
      p.append(x)

  p = ''.join([str(elem) for elem in p])
  
  return p

In [98]:
def conj_verb(verbo_esp,verbo_ladino):
  verbo_phrase = verbo_esp.text
  verbo_inf = verbo_esp.lemma
  if "ar" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
    verbo_ladino = ''.join(verbo_ladino.rsplit('ar',1))
  if "er" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
    verbo_ladino = ''.join(verbo_ladino.rsplit('er',1))
  if "ir" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
    verbo_ladino = ''.join(verbo_ladino.rsplit('ir',1))

  if "ando" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ando"
  elif "iendo" in verbo_phrase:
    verbo_ladino = verbo_ladino+"iendo"
  elif "ado" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ado"
  elif "ido" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ido"
  else:
    suff = get_suffix(verbo_phrase,verbo_inf)
    verbo_ladino = verbo_ladino+suff
  return verbo_ladino

In [97]:
phrase="ayer compré una torta"

In [107]:
def translate(phrase):
  doc = nlp(phrase)
  jud_phrase = ""
  w = ""
  for sent in doc.sentences:
    for word in sent.words:
      flag1 = 0
      español_word = word.text
      español_word = español_word.lower()
      español_word = remove_punctuation(español_word)
      for d in dic:
        if word.lemma == d["src"]:
          ladino_word = d["target"].replace("\n","")
          if word.lemma == ladino_word:
            w = español_word
            break
          if word.upos == "VERB":
            w = conj_verb(word,ladino_word)
          else:
            w = ladino_word
          flag1 = 1
          break
      if flag1 == 0:
          w = judeo_parse(español_word)
      jud_phrase += w.replace("\n","") + " "
  return jud_phrase

In [108]:
translate(phrase)

'ayer marké una torta '

Testing

In [109]:
import pandas as pd

df = pd.read_excel("una-fraze-al-dia_lad-tur-eng-spa.xlsx")

esp=[]
lad_correct = []
lad_trans = []

for a in df.index:
  esp.append(df["Español"][a])
  lad_correct.append(df["Ladino"][a])
  lad_trans.append(translate(df["Español"][a]))

p = {'Español':esp,'Ladino':lad_correct,'Ladino_Trans':lad_trans}

df_1 = pd.DataFrame(p)
df_1.head()

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


,Español,Ladino,Ladino_Trans
0,La habitación del paciente está llena de flore...,La kamareta del hazino esta yena de rozas i bo...,la kamareta de el hazino está yeno de roza i ...
1,Moshe está hablando con la familia y les está ...,Moshe esta avlando kon la famiya i les esta da...,mose está avlando kon la famiya i les está dan...
2,Moshe está muy contento porque el paciente es ...,Moshe esta muy kontente porke el hazino es el ...,mose está muncho orozo por ke el hazino es el ...
3,"La mujer del paciente le trajo a Moshe ""boreka...",La mujer del hazino le trusho a Moshe borekita...,la mujer de el hazino le trajo a mose boreka...
4,"Moshe llevó las ""borekas""a casa y le dió la bu...",Moshe se yevo las borekas a kaza i le dio el b...,mose yevó las borekas a kaza i le dió la b...
